<a href="https://colab.research.google.com/github/CristhianCastro96/BITrepository/blob/main/Desafio_Luflox_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import json
import pandas as pd

# Cargo archivos JSON
with open('/content/Candidate Education.json', 'r') as f:
    education_data = json.load(f)

with open('/content/Candidate Working Experience.json', 'r') as f:
    experience_data = json.load(f)

with open('/content/Job Description.json', 'r') as f:
    job_data = json.load(f)

# Convierto educación en DataFrame
education_df = pd.json_normalize(education_data['education'])

# Convierto experiencia en DataFrame
experience_df = pd.json_normalize(
    experience_data['experiences'],
    'roles',
    ['company', 'positionType', 'positionModality']
)

# Convierto descripción del trabajo en DataFrame
job_df = pd.json_normalize(job_data)

# Muestro los DataFrames para verificar
print("DataFrame de Educación:\n", education_df)
print("\nDataFrame de Experiencia:\n", experience_df)
print("\nDataFrame de Descripción del Trabajo:\n", job_df)


DataFrame de Educación:
   degree     status               institute  \
0  BA/BS  COMPLETED  Universidad Tecmilenio   

                                              career  endDate.seconds  \
0  Mechatronics, Robotics, and Automation Enginee...       1419573600   

   endDate.nanoseconds  startDate.seconds  startDate.nanoseconds  
0                    0         1262584800                      0  

DataFrame de Experiencia:
                                role  \
0                   DevOps Engineer   
1            Azure DevOps Engineer    
2  DevOps Virtualization Specialist   
3         Infrastructure Specialist   

                                    responsibilities  \
0  [Architecting, building, and improving the sca...   
1  [Monitoring and maintaining huge builds of the...   
2  [Creating and deploying new virtual redhat mac...   
3  [Keeping all infrastructure secure, updating a...   

                                         description  startDate.seconds  \
0  Working on a cus

Ahora quiero conocer el tiempo en años de experiencia.

In [46]:
import pandas as pd
from datetime import datetime

# Convierto las columnas de segundos a fechas
experience_df['startDate'] = pd.to_datetime(experience_df['startDate.seconds'], unit='s')
experience_df['endDate'] = pd.to_datetime(experience_df['endDate.seconds'], unit='s')

# Calculo los años de experiencia en cada fila
experience_df['experience_years'] = experience_df.apply(
    lambda row: (row['endDate'] - row['startDate']).days / 365, axis=1
)

# Calculo la experiencia total sumando los años de experiencia de cada posición
total_experience_years = experience_df['experience_years'].sum()

# Muestro el DataFrame con la nueva columna y el total de años de experiencia
print(experience_df[['role', 'company', 'startDate', 'endDate', 'experience_years']])
print(f"Total years of experience: {total_experience_years:.2f} years")


                               role             company           startDate  \
0                   DevOps Engineer              Luflox 2023-01-02 06:00:00   
1            Azure DevOps Engineer          Akvelon INC 2022-03-01 06:00:00   
2  DevOps Virtualization Specialist  NXP Semiconductors 2017-03-01 06:00:00   
3         Infrastructure Specialist          IBM Mexico 2018-05-01 05:00:00   

              endDate  experience_years  
0 2023-12-29 06:00:00          0.989041  
1 2022-11-30 06:00:00          0.750685  
2 2022-03-01 06:00:00          5.002740  
3 2019-05-31 05:00:00          1.082192  
Total years of experience: 7.82 years


Ahora voy a convertir a embedding la descripcion de los trabajos en los que ha estado el candidato, teniendo en cuenta el rol, la descripción y las responsabilidades.

In [53]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Cargo el modelo preentrenado de Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Combino el role, la descripción y las responsabilidades en un solo texto para cada trabajo
experience_df['combined_text'] = (
    experience_df['role'] + " " +
    experience_df['description'] + " " +
    experience_df['responsibilities'].apply(lambda x: ' '.join(x))
)

# Genero los embeddings
embeddings = model.encode(experience_df['combined_text'].tolist())

# Convierto los embeddings en un DataFrame para su análisis
embeddings_df = pd.DataFrame(embeddings)

# Muestro el DataFrame de embeddings
print(embeddings_df.head())


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


        0         1         2         3         4         5         6    \
0 -0.004545 -0.044899  0.011999  0.015242 -0.008560 -0.099734 -0.028823   
1 -0.011244  0.025386  0.033990  0.007177 -0.042514 -0.061415  0.016476   
2 -0.013770  0.025543  0.084357 -0.044327  0.004547 -0.083993 -0.023935   
3 -0.001005  0.075701 -0.042829 -0.010030  0.020374 -0.035992  0.081292   

        7         8         9    ...       374       375       376       377  \
0 -0.031144 -0.000978  0.012843  ...  0.146548  0.042375  0.026663 -0.048898   
1 -0.012981  0.032243  0.108597  ...  0.063074  0.098472  0.009248 -0.062827   
2  0.007329 -0.058098  0.024970  ...  0.059904  0.074595 -0.024459  0.004438   
3  0.019113 -0.053085 -0.050578  ... -0.003757  0.026270  0.068274 -0.025597   

        378       379       380       381       382       383  
0  0.126237  0.114810  0.035420 -0.027398 -0.002313  0.003592  
1  0.053935  0.140336 -0.001346 -0.034449  0.067301  0.016047  
2  0.052823  0.090165 -0.024674

Ahora hago un Embedding del Job description ue incluya las responsabilidades, preferred, role, stack, description y requirements.

In [51]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Cargo el modelo preentrenado de Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Combino las columnas en job_df: ['responsibilities', 'preferred', 'role', 'stack', 'description', 'requirements']

# Función para convertir listas a cadenas
def to_string(x):
    if isinstance(x, list):
        return ' '.join(map(str, x))  # Convertir cada elemento a str y unir
    return str(x)  # Si no es lista, convertir a str

# Combino todos los campos relevantes en un solo texto
job_df['combined_text'] = (
    job_df['responsibilities'].apply(to_string) + " " +
    job_df['preferred'].apply(to_string) + " " +
    job_df['role'].apply(to_string) + " " +
    job_df['stack'].apply(to_string) + " " +
    job_df['description'].apply(to_string) + " " +
    job_df['requirements'].apply(to_string)
)

# Genero los embeddings
embeddings = model.encode(job_df['combined_text'].tolist())

# Convierto los embeddings en un DataFrame para su análisis
embeddings_df = pd.DataFrame(embeddings)

# Muestro el DataFrame de embeddings
print(embeddings_df.head())


        0         1         2         3         4         5         6    \
0 -0.067052 -0.011375 -0.000248  0.008052 -0.015139 -0.076249 -0.028596   

       7         8         9    ...       374       375       376       377  \
0  0.03672 -0.065317 -0.022223  ...  0.056542  0.060545  0.020345 -0.013978   

        378       379       380       381       382       383  
0  0.062176  0.107852  0.059268 -0.077877 -0.028277  0.052238  

[1 rows x 384 columns]


Ahora tomo los resultados de los dos Embeddings y hago un score que determine el porcentaje en el que el candidato hace match con el job description

In [52]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Cargo el modelo preentrenado de Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Genero embeddings para el candidato (experience_df)
candidate_descriptions = experience_df['description'].apply(to_string) + " " + experience_df['responsibilities'].apply(to_string)
candidate_embeddings = model.encode(candidate_descriptions.tolist())

# Genero embeddings para el trabajo (job_df)
job_combined_text = (
    job_df['responsibilities'].apply(to_string) + " " +
    job_df['preferred'].apply(to_string) + " " +
    job_df['role'].apply(to_string) + " " +
    job_df['stack'].apply(to_string) + " " +
    job_df['description'].apply(to_string) + " " +
    job_df['requirements'].apply(to_string)
)
job_embeddings = model.encode(job_combined_text.tolist())

# Calculo la similitud del coseno
match_scores = cosine_similarity(candidate_embeddings, job_embeddings)

# Calculo el porcentaje de coincidencia
# Suponiendo que tomamos el primer candidato y el primer trabajo
match_percentage = match_scores[0][0] * 100

print(f'Porcentaje de coincidencia entre el candidato y el puesto de trabajo: {match_percentage:.2f}%')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Porcentaje de coincidencia entre el candidato y el puesto de trabajo: 62.26%
